<a href="https://colab.research.google.com/github/dteso/AI-Mini-Trainer/blob/main/AI_mini_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio scikit-learn pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import plotly.graph_objects as go

from sklearn.metrics import confusion_matrix
import numpy as np
import plotly.subplots as sp

# Modelos y datasets disponibles
models = {
    "Random Forest": RandomForestClassifier,
    "Logistic Regression": LogisticRegression,
    "SVM": SVC,
    "K-Nearest Neighbors": KNeighborsClassifier,
    "Decision Tree": DecisionTreeClassifier,
    "Gradient Boosting": GradientBoostingClassifier,
    "Naive Bayes": GaussianNB
}

dataset_loaders = {
    "Iris": datasets.load_iris,
    "Wine": datasets.load_wine,
    "Breast Cancer": datasets.load_breast_cancer,
    "Digits": datasets.load_digits  # <- nuevo dataset de clasificación
}

# Variables globales
model_trained = None
feature_names_global = []
target_names_global = []
# trained_models = []
trained_models = {}

def train_model_single(dataset_name, model_name, test_size):
    global model_trained, feature_names_global, target_names_global, trained_models

    data = dataset_loaders[dataset_name]()
    X = data.data
    y = data.target
    feature_names_global = data.feature_names
    target_names_global = data.target_names

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    model = models[model_name]()
    model.fit(X_train, y_train)
    model_trained = model

    trained_models[model_name] = model  # Guarda el modelo por nombre

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=target_names_global)

    # Mostrar primeras 10 filas del dataset como texto tabulado
    sample_data = X[:10]
    sample_target = y[:10]
    preview = "Primeros 10 registros del dataset:\n\n"
    header = "\t".join(feature_names_global) + "\tClase\n"
    preview += header
    for row, label in zip(sample_data, sample_target):
        row_str = "\t".join([f"{val:.2f}" for val in row]) + f"\t{target_names_global[label]}"
        preview += row_str + "\n"

    return f"Modelo entrenado con precisión: {acc:.2f}\n\n{report}\n\n{preview}"



def train_all_models(dataset_name, model_names, test_size):
    data = dataset_loaders[dataset_name]()
    X = data.data
    y = data.target
    global feature_names_global, target_names_global
    feature_names_global = data.feature_names
    target_names_global = data.target_names

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    results = []
    confusion_matrices = {}

    for model_name in model_names:
        model = models[model_name]()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        results.append((model_name, acc))
        confusion_matrices[model_name] = cm

    # Ordenar por precisión descendente
    results.sort(key=lambda x: x[1], reverse=True)

    # Gráfica de precisión
    model_names_sorted = [r[0] for r in results]
    accuracies_sorted = [r[1] for r in results]
    colors = ['purple', 'magenta', 'blue', 'orange', 'green', 'cyan', 'gray']  # Añade más si hace falta
    bar_colors = colors[:len(model_names_sorted)]

    fig_accuracy = go.Figure(
        data=[go.Bar(x=model_names_sorted, y=accuracies_sorted, marker_color=bar_colors)],
        layout=go.Layout(
            title=f'Precisión por modelo - {dataset_name}',
            xaxis_title='Modelos',
            yaxis_title='Precisión',
            yaxis=dict(range=[0, 1])
        )
    )

    # Mostrar primeras 10 filas del dataset como texto tabulado
    sample_data = X[:10]
    sample_target = y[:10]
    log = "Primeros 10 registros del dataset:\n\n"
    header = "\t".join(feature_names_global) + "\tClase\n"
    log += header
    for row, label in zip(sample_data, sample_target):
        row_str = "\t".join([f"{val:.2f}" for val in row]) + f"\t{target_names_global[label]}"
        log += row_str + "\n"

    # Texto de ranking
    log += "\n\nRanking de Modelos por Precisión:\n\n"
    for i, (name, acc) in enumerate(results, 1):
        log += f"{i}. {name} - {acc:.2f}\n"

    # Matrices de confusión en subplots
    cols = 2
    rows = (len(model_names_sorted) + 1) // cols
    fig_cm = sp.make_subplots(rows=rows, cols=cols, subplot_titles=model_names_sorted)

    for idx, model_name in enumerate(model_names_sorted):
        cm = confusion_matrices[model_name]
        row = (idx // cols) + 1
        col = (idx % cols) + 1
        heatmap = go.Heatmap(
            z=cm,
            x=target_names_global,
            y=target_names_global,
            colorscale='Blues',
            showscale=False
        )
        fig_cm.add_trace(heatmap, row=row, col=col)

    fig_cm.update_layout(
        title="Matrices de Confusión por Modelo",
        height=300 * rows,
        width=800
    )

    return log, fig_accuracy, fig_cm


def predict_model_single(input_str):
    global model_trained, target_names_global

    if model_trained is None:
        return "Por favor entrena un modelo primero."

    try:
        # Espera que el usuario introduzca valores separados por coma
        values = list(map(float, input_str.split(',')))
    except Exception as e:
        return "Introduce valores numéricos válidos separados por comas."

    if len(values) != len(feature_names_global):
        return f"Se esperan {len(feature_names_global)} valores, ingresaste {len(values)}."

    prediction = model_trained.predict([values])[0]
    label = target_names_global[prediction]
    return f"Predicción: {label}"

def predict_model_selected(input_str, selected_model_name):
    global target_names_global, feature_names_global

    if selected_model_name not in trained_models:
        return "Modelo no entrenado o no seleccionado."

    model = trained_models[selected_model_name]

    try:
        values = list(map(float, input_str.split(',')))
    except Exception:
        return "Introduce valores numéricos válidos separados por comas."

    if len(values) != len(feature_names_global):
        return f"Se esperan {len(feature_names_global)} valores, ingresaste {len(values)}."

    prediction = model.predict([values])[0]
    label = target_names_global[prediction]
    return f"Predicción: {label}"

def get_trained_models_dropdown():
    if not trained_models:
        return gr.update(choices=[], value=None, interactive=False)
    return gr.update(choices=list(trained_models.keys()), value=list(trained_models.keys())[0], interactive=True)

def update_placeholder(dataset_name):
    data = dataset_loaders[dataset_name]()
    feat = data.feature_names
    # Creamos una cadena con los nombres de las características separados por coma
    placeholder = ", ".join(feat)
    return gr.update(placeholder=placeholder)

model_dropdown_pred = gr.Dropdown(label="Selecciona modelo entrenado", choices=[], interactive=False)

with gr.Blocks(theme=gr.themes.Base(primary_hue="blue", secondary_hue="green")) as demo:
    gr.Markdown("## 🧠 MiniAI Trainer")

    # Pestaña Laboratorio
    with gr.Tab("🧪 Laboratorio"):
      gr.Markdown("### Comparación de Modelos")
      gr.Markdown("""
      Aquí puedes entrenar todos los modelos seleccionados con su correspondiente dataset y visualizar su precisión en una gráfica de barras.
      """)

      with gr.Row():
          dataset_dropdown_lab = gr.Dropdown(label="📁 Selecciona dataset", choices=list(dataset_loaders.keys()), value="Iris")
          model_dropdown_lab = gr.CheckboxGroup(label="Selecciona los modelos", choices=list(models.keys()), value=["Random Forest"])
          test_slider_lab = gr.Slider(0.1, 0.5, value=0.3, label="Tamaño de prueba")

      train_button_lab = gr.Button("Entrenar modelos seleccionados")
      train_output_lab = gr.Textbox(label="🏆 Resultado del entrenamiento (Ranking)", lines=10)
      plot_output_lab = gr.Plot(label="📊 Gráfica de precisión")
      cm_output_lab = gr.Plot(label="📊 Matrices de confusión")

      train_button_lab.click(
          fn=train_all_models,
          inputs=[dataset_dropdown_lab, model_dropdown_lab, test_slider_lab],
          outputs=[train_output_lab, plot_output_lab, cm_output_lab]
      )

    with gr.Tab("⚙️ Entrenamiento"):
        with gr.Row():
            dataset_dropdown = gr.Dropdown(label="Selecciona dataset", choices=list(dataset_loaders.keys()), value="Iris")
            model_dropdown = gr.Dropdown(label="Modelo", choices=list(models.keys()), value="Random Forest")
            test_slider = gr.Slider(0.1, 0.5, value=0.3, label="Tamaño de prueba")

        train_button = gr.Button("Entrenar modelo")
        train_output = gr.Textbox(label="Resultado del entrenamiento", lines=10)

        train_button.click(
            fn=train_model_single,
            inputs=[dataset_dropdown, model_dropdown, test_slider],
            outputs=train_output
        ).then(
            fn=get_trained_models_dropdown,
            inputs=None,
            outputs=model_dropdown_pred
        )

    with gr.Tab("🔍 Predicción"):
        gr.Markdown("Introduce los valores para predecir separados por comas:")

        dataset_dropdown_pred = gr.Dropdown(label="Selecciona dataset", choices=list(dataset_loaders.keys()), value="Iris")
        model_dropdown_pred = gr.Dropdown(label="Selecciona modelo entrenado", choices=[], interactive=True)

        input_text = gr.Textbox(label="Valores", placeholder="Aquí se mostrarán los nombres de las características")
        predict_button = gr.Button("Predecir")
        predict_output = gr.Textbox(label="Resultado de la predicción")

        dataset_dropdown_pred.change(fn=update_placeholder, inputs=[dataset_dropdown_pred], outputs=input_text)
        demo.load(fn=update_placeholder, inputs=[dataset_dropdown_pred], outputs=input_text)
        demo.load(fn=get_trained_models_dropdown, outputs=model_dropdown_pred)

        train_button.click(fn=get_trained_models_dropdown, outputs=model_dropdown_pred)  # 🔁 Actualiza al entrenar

        predict_button.click(fn=predict_model_selected, inputs=[input_text, model_dropdown_pred], outputs=predict_output)


demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8b021cf441b16cf0fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
